In [38]:
# https://www.kaggle.com/code/emirkocak/in-depth-series-sentiment-analysis-w-transformers
import torch
import torch.nn as nn
import torchtext
from torchtext.data.functional import generate_sp_model, load_sp_model, sentencepiece_tokenizer, sentencepiece_numericalizer
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as T
from torch.nn import functional as F
import torch.optim as optim

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import math
from torch.utils.data import Dataset, DataLoader
import torch.utils.data.dataloader as dataloader
import os
import re

In [39]:
root = "/kaggle/input/movies"

In [47]:
# Specify the correct encoding (e.g., 'utf-8', 'latin-1', etc.)
with open(input_file, encoding='utf-8') as f:  # Adjust 'utf-8' to the actual encoding if different
    with open(output_file, "w", encoding='utf-8') as f2:
        for i, line in enumerate(f):
            try:
                text_only = "".join(line.split(",")[:-1])
                f2.write(text_only + "\n")
            except IndexError:
                print(f"Error processing line {i}: {line.strip()}")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 313: invalid continuation byte

In [40]:
with open(os.path.join(root, "reviews.csv")) as f:
        with open(os.path.join("/kaggle/working", "data.txt"), "w") as f2:
            for line in f:
                text_only = "".join(line.split(",")[:-1])
                filtered = re.sub(r'\\|\\n|;', ' ', text_only.replace('"', ' ').replace('\n', ' ')) # Replaces double quotes with a space, and \n with a space
                # Replaces \\, \\n, and; with a space
                # Replaces HTML codes with real characters
                filtered = filtered.replace(' #39;', "'")
                filtered = filtered.replace(' #38;', "&")
                filtered = filtered.replace(' #36;', "$")
                filtered = filtered.replace(' #151;', "-")
                f2.write(filtered.lower() + "\n")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 313: invalid continuation byte

In [10]:
# Generate the SentencePiece tokenizer
# Text tokenizer and detokenizer
# It will tokenize words into subpieces instead of words
# This function will create a set of subtokens to fit the set vocabulary size
# There will always be enough subwords to subtokenize a dataset if you think about it :) -> max 2 length pairs = 26!
# Saved in the home directory
generate_sp_model(os.path.join("/kaggle/working", "data.txt"), vocab_size=30000, model_prefix='/kaggle/working/transformer')

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/kaggle/working/data.txt --model_prefix=/kaggle/working/transformer --vocab_size=30000 --model_type=unigram
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /kaggle/working/data.txt
  input_format: 
  model_prefix: /kaggle/working/transformer
  model_type: UNIGRAM
  vocab_size: 30000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
 

In [36]:
# Class must inherit from Dataset to use DataLoader from torch
class IMDB(Dataset):
    def __init__(self):

        # Reads the file into a pandas DataFrame
        self.df = pd.read_csv(os.path.join(root, "reviews.csv"), names=["Article", "Class"])

        # Replaces empty entries with a space
        self.df.fillna('', inplace=True)

        self.df['Article'] = self.df['Article'].str.replace(r'\\n|\\|\\r|\\r\\n|\n|"', ' ', regex=True)
        self.df['Article'] = self.df['Article'].replace({' #39;': "'", ' #38;': "&", ' #36;': "$", ' #151;': "-"}, regex=True)

    # To use for DataLoader
    def __getitem__(self, index):
        text = self.df.loc[index]["Article"].lower()
        
        class_label = self.df.loc[index]["Class"]

        if class_label == 'positive':
            class_index = 1
        else:
            class_index = 0
            
        return class_index, text

    def __len__(self):
        return len(self.df)

train_dataset = IMDB()
print(len(train_dataset))
print(train_dataset.df.loc[0]["Article"])
train_dataset.df.loc[0]["Class"]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 16697: invalid continuation byte

In [12]:
# Split 90% - 10%
validation_split = 0.9

# Total train examples
n_train_examples = int(len(train_dataset) * validation_split)

# Total validation examples
n_valid_examples = len(train_dataset) - n_train_examples

# Splits them based on values provided
train_data, valid_data = torch.utils.data.random_split(train_dataset, [n_train_examples, n_valid_examples], generator=torch.Generator().manual_seed(42))

In [13]:
# Create dataloaders for the training and testing datasets
# Dataloaders allow for batching, shuffling

batch_size = 128

train_loader = dataloader.DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last = True)

test_loader = dataloader.DataLoader(valid_data, shuffle=True, batch_size=batch_size, drop_last = True)

In [14]:
def yield_tokens(file_path):
    with io.open(file_path, encoding='utf-8') as f:
        # Iterate through each line in the file
        for line in f:
            # Accesses the vocab file, splits the line by tab, and gets the first entry (the actual token)
            # Yield the token from the first column (split by tab)
            yield [line.split("\t")[0]]

# Build a vocabulary from the tokens yielded by the yield_tokens function
    # <pad> is a padding token that is added to the end of a sentence to ensure the length of all sequences in a batch is the same
    # <sos> signals the "Start-Of-Sentence" aka the start of the sequence
    # <eos> signal the "End-Of-Sentence" aka the end of the sequence
    # <unk> "unknown" token is used if a token is not contained in the vocab
# From torchtext library (build_vocab_from_iterator)
# Builds a generator object, that is treated like an iterator
vocab = build_vocab_from_iterator(yield_tokens("/kaggle/working/transformer.vocab"), specials=['<cls>', '<pad>', '<eos>', '<unk>'], special_first=True)

# Set the default index for unknown tokens to the index of the '<unk>' token
vocab.set_default_index(vocab['<unk>'])

In [15]:
# Maximum sequence length for text inputs
max_len = 256

# Data transform to turn text into vocab tokens
text_transform = T.Sequential(
    # Tokenize with pre-existing Tokenizer
    T.SentencePieceTokenizer("/kaggle/working/transformer.model"),
    # Converts the sentences to indices based on given vocabulary
    T.VocabTransform(vocab=vocab),
    # Add <sos> at the beginning of each sentence. 1 because the index for <sos> in the vocabulary is 1 as seen in previous section
    T.AddToken(vocab['<sos>'], begin=True),
    # Crop the sentence if it is longer than the max length minus 2 to accommodate <sos> and <eos> tokens
    T.Truncate(max_seq_len=max_len-2),
    # Add <eos> at the end of each sentence. 2 because the index for <eos> in the vocabulary is 2 as seen in previous section
    T.AddToken(vocab['<eos>'], begin=False),
    # Convert the list of lists to a tensor. This will also pad a sentence with the <pad> token if it is shorter than the max length.
    # This ensures all sentences are the same length!
    T.ToTensor(padding_value=vocab['<pad>']),
    # Pad the sequence to ensure it's exactly max_len tokens long
    T.PadTransform(max_length=max_len, pad_value=0),
)

In [16]:
class TokenDrop(nn.Module):
    """ For a batch of tokens indices, randomly replace a non-specical token with <pad>.
    prob (float): probability of dropping a token
    pad_token (int): index for the <pad> token
    num_special (int): Number of special tokens, assumed to be at the start of the vocab
    """

    def __init__(self, prob=0.1, pad_token=0, num_special=4):
        self.prob = prob
        self.num_special = num_special
        self.pad_token = pad_token

    def __call__(self, sample):
        # Randomly sample a bernoulli distribution with p = prob
        # Create a mask where 1 means we will replace that token
        # Discrete probability distribution
        # Here we want to treat the ones as the indexes to drop
        mask = torch.bernoulli(self.prob * torch.ones_like(sample)).long()
        
        # Only replace if the token is not a special token
        # Ones or zeros. If cannot drop, 0, if can drop, 1
        can_drop = (sample >= self.num_special).long()
        # Multiply together to get the corresponding tokens to be dropped and not dropped
        # Here, 1 represents drop, 0 represents do not drop
        mask = mask * can_drop

        # Make a mask of pad_token to use for replacing dropped indices with the pad_token
        replace_with = (self.pad_token * torch.ones_like(sample)).long()
        """ Sample is the original sample
        The mask indicates what tokens can be replaced (0 to not be replaced, 1 to be replaced)
        Replace_with is a list of of the pad_token tokens
        Here, (1-mask) creates the complement mask. (now, 0 indicates drop, 1 indicates to not drop)
        1-1 = 0, 1-0 = 0
        Multiplying by sample, retains the original tokens that are not to be kept, and applies the mask on the sample
        Here, mask * replace_with does elementwise multiplication and adds the corresponding pad_token to the tokens replaced
        """
        sample_out = (1 - mask) * sample + mask * replace_with
        
        return sample_out

In [17]:
class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        # Get the device of the input tensor
        device = x.device
        
        # Calculate half of the hidden size
        half_dim = self.dim // 2
        
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = x[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        
        return emb

In [18]:
class NanoTransformer(nn.Module):
    """
        This class implements a simplified Transformer model for sequence classification. 
        It uses an embedding layer for tokens, sinusoidal positional embeddings, 
        a Transformer, and a Linear layer.
        
        num_emb: The number of unique tokens in the vocabulary. (vocab_size)
        output_size: The size of the output layer (number of classes). (4)
        hidden_size: The dimension of the hidden layer in the Transformer block (default: 128)
        num_heads: The number of heads in the multi-head attention layer (default: 4).
    """
    def __init__(self, num_emb, output_size, hidden_size=128, num_heads=4):
        
        # Inherits from nn.Module's attributes
        super(NanoTransformer, self).__init__()

        # Create an embedding for each token
        self.embedding = nn.Embedding(num_emb, hidden_size) # (vocab_size, 128)
        
        # Scaling down the embedding weights
        self.embedding.weight.data = 0.001 * self.embedding.weight.data
        
        # Positional embedding
        self.pos_emb = SinusoidalPosEmb(hidden_size)

        # Multi-head attention
        self.multihead_attn = nn.MultiheadAttention(hidden_size, num_heads = num_heads, batch_first = True)

        # Linear layer
        self.mlp = nn.Sequential(nn.Linear(hidden_size, hidden_size), # (batch_size, 128, 128)
                                 nn.LayerNorm(hidden_size), # (batch_size, 128, 128)
                                 nn.ELU(), # (batch_size, 128, 128)
                                 nn.Linear(hidden_size, hidden_size)) # (batch_size, 128, 128)
        
        self.fc_out = nn.Linear(hidden_size, output_size) # (batch_size, 128, 128)

    def forward(self, input_seq):
        # batch_size, time_steps
        batch_size, l = input_seq.shape # (32, 160)

        input_embs = self.embedding(input_seq) # (32, 160) -> (32, 160, 128)
        
        # Add a unique embedding to each token embedding depending on it's position in the sequence
        seq_indx = torch.arange(l) # (160)
        
        pos_emb = self.pos_emb(seq_indx).reshape(1, l, -1).expand(batch_size, l, -1) # (1, 160, 128) -> (32, 160, 128)
        
        pos_emb = pos_emb.to(device)
        
        embs = input_embs + pos_emb # (32, 160, 128) + (32, 160, 128)
        
        
        output, attn_map = self.multihead_attn(embs, embs, embs) # (32, 160, 128)
        
        output = self.mlp(output) # (32, 160, 128)

        return self.fc_out(output), attn_map # (32, 160, 4)

In [19]:
# Set the device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

learning_rate = 1e-4

nepochs = 20

hidden_size = 256

output_size = 2

num_heads = 4

tf_classifier = NanoTransformer(num_emb=len(vocab), output_size=2, hidden_size=hidden_size, num_heads=num_heads).to(device)

# Initialize the optimizer
optimizer = optim.Adam(tf_classifier.parameters(), lr=learning_rate, weight_decay=1e-4)

# Cosine annealing scheduler to decay the learning rate
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=nepochs, eta_min=0)

loss_fn = nn.CrossEntropyLoss()
td = TokenDrop(prob=0.4)

training_loss_list = []
test_loss_list = []
training_acc_list = []
test_acc_list = []

In [20]:
device

device(type='cuda')

In [21]:
# Number of model parameters
num_model_params = 0
for param in tf_classifier.parameters():
    num_model_params += param.flatten().shape[0]

print("-This Model Has %d (Approximately %d Million) Parameters!" % (num_model_params, num_model_params//1e6))

-This Model Has 8076546 (Approximately 8 Million) Parameters!


In [ ]:
# Initialize training and testing accuracy lists
train_acc = 0
test_acc = 0

# Loop over each epoch
for epoch in range(nepochs):
    
    tf_classifier.train()
    
    print("training")
    
    train_acc_count = 0
    test_acc_count = 0
    train_steps = 0

    # Loop over each batch in the training dataset
    for labels, texts in train_loader:
        batch_size = labels.shape[0]
        
        # Transform the text to tokens and move to the GPU
        text_tokens = text_transform(list(texts)).to(device)
        labels = labels.to(device)
        
        # TokenDrop
        text_tokens = td(text_tokens).to(device)
        
        # Get the model predictions
        pred, _ = tf_classifier(text_tokens)
        
        # Compute the loss using cross-entropy loss
        loss = loss_fn(pred[:, 0, :], labels)
        
        # Backpropagation and optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
#         print(loss)
        
        # Log the training loss
        training_loss_list.append(loss.item())
        
        # Update training accuracy
        train_acc_count += (pred[:, 0, :].argmax(1) == labels).sum().item()
        train_steps += batch_size
    
    # Calculate average training accuracy
    train_acc = (train_acc_count / train_steps)
    training_acc_list.append(train_acc)
    
    # Update learning rate
    lr_scheduler.step()
    
    # Set the model to evaluation mode
    tf_classifier.eval()
    
    print("evaluating")
    train_acc_count = 0
    test_acc_count = 0
    test_steps = 0
    
    # Loop over each batch in the testing dataset
    with torch.no_grad():
        
        for labels, texts in test_loader:
            batch_size = labels.shape[0]
            
            # Transform the text to tokens and move to the GPU
            text_tokens = text_transform(list(texts)).to(device)
            labels = labels.to(device)
            
            # Get the model predictions
            pred, _ = tf_classifier(text_tokens)
            
            # Compute the loss using cross-entropy loss
            loss = loss_fn(pred[:, 0, :], labels)
            test_loss_list.append(loss.item())
            
            # Update testing accuracy
            test_acc_count += (pred[:, 0, :].argmax(1) == labels).sum().item()
            test_steps += batch_size
        
        # Calculate average testing accuracy
        test_acc = (test_acc_count / test_steps)
        test_acc_list.append(test_acc)

    # Print out the results for this epoch
    print(f'Epoch {epoch+1}/{nepochs}')
    print(f'Training Accuracy: {train_acc*100:.2f}%')
    print(f'Testing Accuracy: {test_acc*100:.2f}%')

training
evaluating
Epoch 1/20
Training Accuracy: 50.36%
Testing Accuracy: 50.94%
training
evaluating
Epoch 2/20
Training Accuracy: 51.47%
Testing Accuracy: 62.62%
training
evaluating
Epoch 3/20
Training Accuracy: 68.04%
Testing Accuracy: 80.71%
training
evaluating
Epoch 4/20
Training Accuracy: 80.15%
Testing Accuracy: 82.95%
training
evaluating
Epoch 5/20
Training Accuracy: 83.25%
Testing Accuracy: 84.62%
training
evaluating
Epoch 6/20
Training Accuracy: 84.82%
Testing Accuracy: 84.76%
training
